<a href="https://colab.research.google.com/github/yyx462/ML/blob/main/%E2%80%9C498Project_2_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np                   # advanced math library
import matplotlib.pyplot as plt      # MATLAB like plotting routines
import random                        # for generating random numbers

import tensorflow as tf
from tensorflow import keras
import pandas as pd

from keras.models import Sequential  # Model type to be used

from keras.layers.core import Dense, Dropout, Activation # Types of layers to be used in our model
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras import regularizers
from keras.utils import np_utils                         # NumPy related tools
from keras.callbacks import EarlyStopping
from keras.layers.core.flatten import Flatten
from keras.layers.pooling import MaxPool2D
from sklearn.model_selection import train_test_split
from numpy import save


# Load Data

In [ ]:
# (X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
X = np.load('training_images.npy')
y = np.load('training_labels.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (45000, 28, 28)
y_train shape (45000,)
X_test shape (15000, 28, 28)
y_test shape (15000,)


In [ ]:
# print("X_train[0] shape", X_train[0].shape)
# print("y_train shape", y_train.shape)
# print("X_test shape", X_test.shape)
# print("y_test shape", y_test.shape)

# Process Input Vector

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [ ]:
nb_classes = 10

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# Build the model

In [ ]:
callBack = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [ ]:

model = Sequential()
#convolution layers and pooling layers
model.add(Conv2D(filters=32, padding='same', kernel_size=(3,3), activation='relu', input_shape=(28,28,1), kernel_regularizer=regularizers.L2(1e-4)))
model.add(BatchNormalization()),
model.add(MaxPool2D((3,3)))
model.add(BatchNormalization()),
model.add(Conv2D(filters=64, padding='same', kernel_size=(4,4), activation='relu', kernel_regularizer=regularizers.L2(1e-4)))
model.add(BatchNormalization()),
model.add(MaxPool2D((3,3)))
model.add(BatchNormalization()),

model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.L2(1e-4)))
model.add(Dropout(0.2))
model.add(Dense(nb_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_split=0.2, verbose=1, callbacks=callBack)

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

469/469 [==============================] - 6s 12ms/step - loss: 0.3769 - accuracy: 0.9128
Test loss: 0.376906156539917
Test accuracy: 0.9128000140190125


In [ ]:
X_out = np.load('test_images.npy')
#X_out = X_out.reshape(10000, 784)   # reshape 10,000 28 x 28 matrices into 10,000 784-length vectors.
X_out = X_out.astype('float32')   # change integers to 32-bit floating point numbers
X_out /= 255
predicted_classes_probabilities = model.predict(X_out)
predicted_classes = np.argmax(predicted_classes_probabilities,axis=1)
df={'LABEL':predicted_classes}
df=pd.DataFrame(data=df)
df['LABEL'].to_csv('output.csv',index_label='ID')